In [54]:
import networkx as nx
import plotly.graph_objs as go

def read_graph_from_file(file_path):
    with open(file_path, 'r') as f:
        n, m = map(int, f.readline().strip().split())  # Use strip() to remove any leading/trailing whitespace
        G = nx.Graph()
        
        for i in range(n):
            G.add_node(i, label=f'Node {i}')
        
        edge_count = 0  # Keep track of edges added
        for line in f:
            if line.strip():  # Check if the line is not empty
                start, end = map(int, line.strip().split())
                G.add_edge(start, end)
                edge_count += 1
                if edge_count >= m:  # Stop if we've read the expected number of edges
                    break
    
    return G

def create_plotly_graph(G):
    pos = nx.spring_layout(G)
    edge_trace = []
    node_trace = []

    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_trace.append(go.Scatter(
            x=[x0, x1, None], 
            y=[y0, y1, None],
            line=dict(width=2, color='#888'),
            hoverinfo='none',
            mode='lines'))

    for node in G.nodes():
        x, y = pos[node]
        node_trace.append(go.Scatter(
            x=[x],
            y=[y],
            text=[str(node)],  # Node number inside the node
            mode='markers+text',
            hoverinfo='text',
            textposition='middle center',  # Center the node label
            marker=dict(
                size=20,
                color='lightblue',
                line_width=2)))

    fig = go.Figure(data=edge_trace + node_trace,
                    layout=go.Layout(
                        title='<br>Interactive Graph Visualization',
                        titlefont_size=16,
                        showlegend=False,
                        hovermode='closest', 
                        margin=dict(b=0, l=0, r=0, t=40),
                        xaxis=dict(showgrid=False, zeroline=False),
                        yaxis=dict(showgrid=False, zeroline=False)))
    
    fig.show()

# Example usage
G = read_graph_from_file('graph_input.txt')
create_plotly_graph(G)


In [55]:

import matplotlib.colors as mcolors

def find_and_color_connected_components(G, colors):
    connected_components = list(nx.connected_components(G))
    component_color_map = {}

    for i, component in enumerate(connected_components):
        color = colors[i % len(colors)]  # Cycle through the provided colors
        for node in component:
            component_color_map[node] = color

    return component_color_map

def find_connected_components(G):
    return list(nx.connected_components(G))

def assign_colors_to_components(connected_components, colors):
    
    component_color_map = {}
    
    for i, component in enumerate(connected_components):
        color = colors[i % len(colors)]
        for node in component:
            component_color_map[node] = color
    
    return component_color_map

def create_plotly_graph_color(G, component_color_map):
    pos = nx.spring_layout(G)  # Get node positions using spring layout
    edge_trace = []
    node_trace = []

    # Create edge traces
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_trace.append(go.Scatter(
            x=[x0, x1, None], 
            y=[y0, y1, None],
            line=dict(width=2, color='#888'),
            hoverinfo='none',
            mode='lines'))

    
    for node in G.nodes():
        x, y = pos[node]
        node_trace.append(go.Scatter(
            x=[x],
            y=[y],
            text=[str(node)],  # Node number inside the node
            mode='markers+text',
            hoverinfo='text',
            textposition='middle center',  # Center the node label
            marker=dict(
                size=20,
                color=component_color_map[node],  # Color based on the connected component
                line_width=2),textfont=dict(color='white')))

    # Create the figure
    fig = go.Figure(data=edge_trace + node_trace,
                    layout=go.Layout(
                        title='<br>Interactive Graph Visualization',
                        titlefont_size=16,
                        showlegend=False,
                        hovermode='closest', 
                        margin=dict(b=0, l=0, r=0, t=40),
                        xaxis=dict(showgrid=False, zeroline=False),
                        yaxis=dict(showgrid=False, zeroline=False)))
    
    fig.show()

colors = list(mcolors.TABLEAU_COLORS.values())


connected_components = find_connected_components(G)


component_color_map = assign_colors_to_components(connected_components, colors)


create_plotly_graph_color(G,component_color_map)


In [56]:
def get_path_connected(connected_components, graph):
    component_color_map = {}
    non_path_connected_components = []
    # Define colors
    red_color = 'red'
    other_color = 'blue'  # You can replace 'blue' with any other color
    
    # Iterate through each connected component
    for i, component in enumerate(connected_components):
        if is_path_connected(component, graph):  # Check if the component is path-connected
            color = red_color  # Color path-connected components red
        else:
            color = other_color  # Color other components with a different color
            non_path_connected_components.append(component)
        
        for node in component:
            component_color_map[node] = color
    
    return component_color_map,non_path_connected_components

def is_path_connected(component, graph):
    degree_one_nodes = [node for node in component if len(graph[node]) == 1]
    if len(component) == 1:
        return True
    
    if len(degree_one_nodes) < 2:
        return False  
    

    visited = set()
    
    
    start_node = degree_one_nodes[0]
    
    def dfs(node):
        visited.add(node)
        for neighbor in graph[node]:
            if neighbor not in visited:
                dfs(neighbor)
    
    dfs(start_node)
    
    
    return all(node in visited for node in component)

component_color_map,non_path_connected_components = get_path_connected(connected_components, G)

create_plotly_graph_color(G, component_color_map)



In [57]:
print("Non-path-connected components:", non_path_connected_components)

Non-path-connected components: [{0, 1, 2, 3, 6, 7, 8, 11, 12, 13, 16, 17, 18}, {9, 19, 4, 14}]
